In [1]:
!pip install boto3

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import boto3, botocore, configparser, json

## 0. Delete table : Notes

In [3]:
dyn_resource = boto3.resource('dynamodb')
table = dyn_resource.Table('Notes')
table.delete()
print(f"Deleting Notes table...")
table.wait_until_not_exists()

Deleting Notes table...


## 1. Creating an Amazon DynamoDB table : createTable.py

In [4]:
def createTable(ddbClient, tableDefinition):
    response = ddbClient.create_table(
        AttributeDefinitions=[
            {
                'AttributeName': tableDefinition["partitionKey"],
                'AttributeType': 'S',
            },
            {
                'AttributeName': tableDefinition["sortKey"],
                'AttributeType': 'N',
            },
        ],
        KeySchema=[
            {
                'AttributeName': tableDefinition["partitionKey"],
                'KeyType': 'HASH',
            },
            {
                'AttributeName': tableDefinition["sortKey"],
                'KeyType': 'RANGE',
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': int(tableDefinition["readCapacity"]),
            'WriteCapacityUnits': int(tableDefinition["writeCapacity"]),
        },
        TableName=tableDefinition["tableName"]
    )
    return response

In [5]:
def waitForTableCreation(ddbClient, tableName):
    waiter = ddbClient.get_waiter('table_exists')
    waiter.wait(
        TableName=tableName
    )

In [6]:
def getTableInfo(ddbClient, tableName):
    response = ddbClient.describe_table(
        TableName=tableName
    )
    return response

In [7]:
def readConfig():
    config = configparser.ConfigParser()
    config.read('./config.ini')
    return config['DynamoDB']

In [8]:
ddbClient = boto3.client('dynamodb')

In [9]:
config = readConfig()

In [10]:
tableDefinition = {
    'tableName': config["tableName"],
    'partitionKey': config["partitionKey"],
    'sortKey': config["sortKey"],
    'readCapacity': config["readCapacity"],
    'writeCapacity': config["writeCapacity"]
    }

In [11]:
tableDefinition

{'tableName': 'Notes',
 'partitionKey': 'UserId',
 'sortKey': 'NoteId',
 'readCapacity': '1',
 'writeCapacity': '1'}

In [12]:
print(f"Creating an Amazon DynamoDB table: {config['tableName']} with a partition key: {config['partitionKey']} and sort key: {config['sortKey']}")
creationResponse = createTable(ddbClient, tableDefinition)

Creating an Amazon DynamoDB table: Notes with a partition key: UserId and sort key: NoteId


## 2. 테이블에 데이터 로드 : loadData.py

In [21]:
import boto3, botocore, configparser, json

In [22]:
def putNote(table, note):
    print("loading note " + str(note))    
    table.put_item(
        Item={
            'UserId': note["UserId"],
            'NoteId': int(note["NoteId"]),
            'Note': note["Note"]
        }
    )

In [23]:
ddbResource = boto3.resource('dynamodb')

In [24]:
tableName = config['tableName']
jsonFileName = config['sourcenotes']

# Opening JSON file
f = open(jsonFileName)

print(f"Loading {tableName} table with data from file {jsonFileName}")
# Load json object from file
notes = json.load(f)

# Create dynamodb table resource
table = ddbResource.Table(tableName)

# Iterating through the notes and putting them in the table
for n in notes:
    putNote(table, n)

# Closing the JSON file
f.close()
print("Finished loading notes from the JSON file")

Loading Notes table with data from file ./notes.json
loading note {'UserId': 'testuser', 'NoteId': '001', 'Note': 'hello'}
loading note {'UserId': 'testuser', 'NoteId': '002', 'Note': 'this is my first note'}
loading note {'UserId': 'newbie', 'NoteId': '001', 'Note': 'Free swag code: 1234'}
loading note {'UserId': 'newbie', 'NoteId': '002', 'Note': 'I love DynamoDB'}
loading note {'UserId': 'student', 'NoteId': '001', 'Note': 'DynamoDB is NoSQL'}
loading note {'UserId': 'student', 'NoteId': '002', 'Note': 'A DynamoDB table is schemaless'}
loading note {'UserId': 'student', 'NoteId': '003', 'Note': 'PartiQL is a SQL compatible language for DynamoDB'}
loading note {'UserId': 'student', 'NoteId': '005', 'Note': 'Maximum size of an item is ____ KB ?'}
loading note {'UserId': 'student', 'NoteId': '004', 'Note': 'I love DyDB'}
Finished loading notes from the JSON file


## 3. 파티션 키 및 프로젝션을 사용하여 데이터 쿼리 : loadData.py

In [25]:
import boto3, botocore, json, decimal, configparser
from boto3.dynamodb.conditions import Key, Attr
from boto3.dynamodb.types import TypeDeserializer

In [26]:
config = readConfig()
tableName = config['tableName']
UserId = config['queryUserId']

In [27]:
def queryNotesByPartitionKey(ddbClient, tableName, qUserId):   
    response = ddbClient.query(
        TableName=tableName,
        KeyConditionExpression='UserId = :userId',
        ExpressionAttributeValues={
            ':userId': {"S": qUserId}
        },
        ProjectionExpression="NoteId, Note"
    )
    return response["Items"]

In [28]:
## Utility methods
def printNotes(notes):
    if isinstance(notes, list):
        for note in notes:
            print(
                json.dumps(
                    {key: TypeDeserializer().deserialize(value) for key, value in note.items()},
                    cls=DecimalEncoder
                )
            )

In [29]:
# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        if isinstance(o, set):  # <---resolving sets as lists
            return list(o)
        return super(DecimalEncoder, self).default(o)

In [30]:
ddbClient = boto3.client('dynamodb')

print(f"\n************\nQuerying for notes that belong to user {UserId}...\n")
printNotes(queryNotesByPartitionKey(ddbClient, tableName, UserId))


************
Querying for notes that belong to user student...

{"Note": "DynamoDB is NoSQL", "NoteId": "1"}
{"Note": "A DynamoDB table is schemaless", "NoteId": "2"}
{"Note": "PartiQL is a SQL compatible language for DynamoDB", "NoteId": "3"}
{"Note": "I love DyDB", "NoteId": "4"}
{"Note": "Maximum size of an item is ____ KB ?", "NoteId": "5"}


## 4: Paginator를 사용하여 테이블 스캔 : paginateData.py

In [31]:
import boto3, botocore, json, decimal, configparser
from boto3.dynamodb.conditions import Key, Attr
from boto3.dynamodb.types import TypeDeserializer

In [32]:
def queryAllNotesPaginator(ddbClient, tableName, pageSize): 
    # Create a reusable Paginator
    paginator = ddbClient.get_paginator('scan')

    # Create a PageIterator from the Paginator
    page_iterator = paginator.paginate(
        TableName=tableName,
        PaginationConfig={
            'PageSize': pageSize
        })

    pageNumber = 0
    for page in page_iterator:
        if page["Count"] > 0:
            pageNumber += 1
            print("Starting page " + str(pageNumber))
            printNotes(page['Items'])
            print("End of page " + str(pageNumber) + "\n")

In [33]:
config = readConfig()
tableName = config['tableName']
pageSize = config['pageSize']

ddbClient = boto3.client('dynamodb')

print("\n************\nScanning with pagination...\n")
queryAllNotesPaginator(ddbClient, tableName, pageSize)


************
Scanning with pagination...

Starting page 1
{"Note": "hello", "UserId": "testuser", "NoteId": "1"}
{"Note": "this is my first note", "UserId": "testuser", "NoteId": "2"}
{"Note": "DynamoDB is NoSQL", "UserId": "student", "NoteId": "1"}
End of page 1

Starting page 2
{"Note": "A DynamoDB table is schemaless", "UserId": "student", "NoteId": "2"}
{"Note": "PartiQL is a SQL compatible language for DynamoDB", "UserId": "student", "NoteId": "3"}
{"Note": "I love DyDB", "UserId": "student", "NoteId": "4"}
End of page 2

Starting page 3
{"Note": "Maximum size of an item is ____ KB ?", "UserId": "student", "NoteId": "5"}
{"Note": "Free swag code: 1234", "UserId": "newbie", "NoteId": "1"}
{"Note": "I love DynamoDB", "UserId": "newbie", "NoteId": "2"}
End of page 3



## 5. 테이블의 항목 업데이트 : updateItem.py

In [34]:
import boto3, botocore, configparser

In [35]:
def updateNewAttribute(ddbClient, tableName, qUserId, qNoteId):    
    ## TODO  : Add code to set an 'Is_Incomplete' flag to 'Yes' for the note that matches the 
    ## provided function parameters
    response = ddbClient.update_item(
        TableName=tableName,
        Key={
            'UserId': {'S': qUserId},
            'NoteId': {'N': str(qNoteId)}
        },
        ReturnValues='ALL_NEW',
        UpdateExpression='SET Is_Incomplete = :incomplete',
        ExpressionAttributeValues={
            ':incomplete': {'S': 'Yes'}
        }
    )
    return response['Attributes']

In [36]:
def updateExistingAttributeConditionally(ddbClient, tableName, qUserId, qNoteId, notePrefix):
    try:
        ## TODO  Add code to update the Notes attribute for the note that matches 
        # the passed function parameters only if the 'Is_Incomplete' attribute is 'Yes'
        
        notePrefix += ' 400 KB'
        response = ddbClient.update_item(
            TableName=tableName,
            Key={
                'UserId': {'S': qUserId},
                'NoteId': {'N': str(qNoteId)}
            },
            ReturnValues='ALL_NEW',
            UpdateExpression='SET Note = :NewNote, Is_Incomplete = :new_incomplete',
            ConditionExpression='Is_Incomplete = :old_incomplete',
            ExpressionAttributeValues={
                ':NewNote': {'S': notePrefix},
                ':new_incomplete': {'S': 'No'},
                ':old_incomplete': {'S': 'Yes'}
            }
        )
        
        return response['Attributes']
    except botocore.exceptions.ClientError as err:
        if err.response['Error']['Code'] == 'ConditionalCheckFailedException':
            return "Sorry, your update is invalid mate!"
        else:
            return err.response['Error']['Message']


In [37]:
config = readConfig()
tableName = config['tableName']
qUserId = config['queryUserId']
qNoteId = config['queryNoteId']
notePrefix = config['notePrefix']

ddbClient = boto3.client('dynamodb')

print("\nUpdating the note flag for remediation...\n")
print(updateNewAttribute(ddbClient, tableName, qUserId, qNoteId))


Updating the note flag for remediation...

{'Note': {'S': 'Maximum size of an item is ____ KB ?'}, 'UserId': {'S': 'student'}, 'NoteId': {'N': '5'}, 'Is_Incomplete': {'S': 'Yes'}}


In [38]:
print("\nRemediating the marked note...\n")
print(updateExistingAttributeConditionally(ddbClient, tableName, qUserId, qNoteId, notePrefix))


Remediating the marked note...

{'Note': {'S': 'The maximum item size in DynamoDB is 400 KB'}, 'UserId': {'S': 'student'}, 'NoteId': {'N': '5'}, 'Is_Incomplete': {'S': 'No'}}


##  6. DynamoDB용 PartiQL(SQL 호환 쿼리 언어) 사용 : partiQL.py


In [39]:
import boto3, botocore, json, decimal, configparser
from boto3.dynamodb.conditions import Key, Attr
from boto3.dynamodb.types import TypeDeserializer

In [40]:
def querySpecificNote(ddbClient, tableName, qUserId, qNoteId):    
    response = ddbClient.execute_statement(
        Statement="SELECT * FROM " + tableName + " WHERE UserId = ? AND NoteId = ?",
        Parameters=[
            {"S": qUserId},
            {"N": str(qNoteId)}
        ]
    )    
    return response["Items"]

In [41]:
config = readConfig()
tableName = config['tableName']
UserId = config['queryUserId']
NoteId = config['queryNoteId']

ddbClient = boto3.client('dynamodb')

print(f"\n************\nQuerying for note {NoteId} that belongs to user {UserId}...\n")
printNotes(querySpecificNote(ddbClient, tableName, UserId, NoteId))


************
Querying for note 5 that belongs to user student...

{"Note": "The maximum item size in DynamoDB is 400 KB", "UserId": "student", "NoteId": "5", "Is_Incomplete": "No"}
